<a href="https://www.kaggle.com/code/ryancardwell/chronosorcav10?scriptVersionId=271405156" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# ChronosOrca v3 — Hybrid Neural–Symbolic ARC Solver (Contest-Safe)
**Design:** Kaggle-friendly, deterministic seeds, verbose logs, local metrics.

In [1]:

import os, sys, json, time, math, random, logging, itertools, statistics
from dataclasses import dataclass, field
from typing import List, Dict, Tuple, Optional, Any, Callable
from collections import defaultdict, Counter, deque
from pathlib import Path
import numpy as np

try:
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    import torch.optim as optim
    TORCH = True
except Exception:
    TORCH = False

try:
    from scipy.optimize import linear_sum_assignment
    from scipy.ndimage import label
    SCIPY = True
except Exception:
    SCIPY = False

logging.basicConfig(level=logging.INFO, format="%(asctime)s | %(levelname)5s | %(message)s")
log = logging.getLogger("chronosorca")

CONFIG = {
    "SEED": 1337,
    "TIME_PER_TASK_S": 10.0,
    "BEAM_WIDTH": 16,
    "BEAM_DEPTH": 6,
    "FAMILY_CAPS": {"geometric": 2, "color": 3, "size": 2, "object": 3, "pattern": 2},
    "RELAXED_OK": True,
    "RELAXED_MIN_IOU": 0.90,
    "BG": 0,
    "VETO_THRESH": 0.35,
    "VETO_FEATS": 32,
    "VETO_HIDDEN": 64,
    "TTT_STEPS": 12,
    "TTT_LR": 1e-3,
    "MAX_PERIOD": 12,
    "MAX_GRID_SIDE": 60,
    "ATTEMPTS_PER_TEST": 2,
    "PRINT_SAMPLE_GRIDS": False,
    "METRIC_FLUSH_EVERY": 1,
}

random.seed(CONFIG["SEED"])
np.random.seed(CONFIG["SEED"])
if TORCH:
    torch.manual_seed(CONFIG["SEED"])

class MetricLog:
    def __init__(self):
        self.rows = []
        self.start = time.time()
    def log(self, **kwargs):
        row = {"t": round(time.time() - self.start, 3)}
        row.update(kwargs)
        self.rows.append(row)
        msg = " | ".join(f"{k}={v}" for k,v in row.items())
        log.info(f"[metrics] {msg}")
    def summary(self):
        agg = defaultdict(list)
        for r in self.rows:
            for k,v in r.items():
                if k == "t": continue
                try:
                    agg[k].append(float(v))
                except:
                    pass
        return {k: round(sum(v)/max(1,len(v)),4) for k,v in agg.items()}

METRICS = MetricLog()

def grid_to_np(g):
    if not g or not isinstance(g[0], list): return np.zeros((1,1), dtype=np.uint8)
    arr = np.array(g, dtype=np.uint8)
    arr[arr>9]=9; arr[arr<0]=0
    return arr

def np_to_grid(a):
    a = np.asarray(a, dtype=np.int16)
    a[a>9]=9; a[a<0]=0
    return a.astype(int).tolist()

def same_shape(a,b):
    return a.shape == b.shape

def safe_iou(a,b):
    if not same_shape(a,b): return 0.0
    return float((a==b).sum())/max(1,a.size)

def palette(g):
    return sorted(list(np.unique(g)))


In [2]:
# === HOTFIX: robust grid <-> numpy, safe train/test parsing, and minor guards ===
import numpy as np

def grid_to_np(g):
    """Robustly convert ARC grid (list[list[int]] or np.ndarray) -> np.uint8 2D array in [0..9]."""
    if isinstance(g, np.ndarray):
        arr = g
    elif isinstance(g, list):
        # Handle empty or ragged lists
        if len(g) == 0 or not isinstance(g[0], list):
            return np.zeros((1, 1), dtype=np.uint8)
        arr = np.array(g, dtype=np.int32)
    else:
        return np.zeros((1, 1), dtype=np.uint8)

    # Force 2D
    if arr.ndim == 0:
        arr = np.zeros((1, 1), dtype=np.int32)
    elif arr.ndim == 1:
        side = int(np.ceil(np.sqrt(arr.size))) or 1
        pad = side * side - arr.size
        if pad > 0:
            arr = np.pad(arr, (0, pad), constant_values=0)
        arr = arr.reshape(side, side)
    elif arr.ndim > 2:
        arr = arr.squeeze()
        if arr.ndim != 2:
            # Fall back to 1x1 if still not 2D
            arr = np.array([[int(arr.flat[0]) if arr.size else 0]])

    # Clamp and cast
    arr = np.clip(arr, 0, 9).astype(np.uint8)
    return arr

def numpy_to_grid(arr):
    """np.ndarray (any dtype/shape) -> list[list[int]] clamped to [0..9], 2D."""
    a = grid_to_np(arr)  # re-use robustness
    return a.tolist()

def safe_get_train(task):
    """Return normalized train pairs with 'input','output' as 2D np.uint8."""
    out = []
    for p in task.get("train", []):
        # Some datasets store np arrays already
        g_in = grid_to_np(p.get("input", []))
        g_out = grid_to_np(p.get("output", []))
        out.append({"input": g_in, "output": g_out})
    return out

def safe_get_test_input(task, k=0):
    """Return kth test input as 2D np.uint8."""
    ti = task.get("test", [])
    if not ti:
        return np.zeros((1, 1), dtype=np.uint8)
    return grid_to_np(ti[k].get("input", []))

# ---- Small safety shims for downstream code that called old helpers ----
# If your code used grid_size(grid_as_list), it will still work via grid_to_np.
def grid_size(g):
    a = grid_to_np(g)
    return int(a.shape[0]), int(a.shape[1])

# === PATCH CALL SITES (minimal): replace in your code ===
# OLD:
# train = [{"input":grid_to_np(p["input"]), "output":grid_to_np(p["output"])} for p in task.get("train",[])]
# test_in = grid_to_np(task["test"][0]["input"])
# NEW:
# train = safe_get_train(task)
# test_in = safe_get_test_input(task, 0)

# Optional: quick self-test to ensure ambiguity error is gone
_g_examples = [[], [[1,2],[3,4]], np.array([[5,6],[7,8]], dtype=np.int32)]
for _g in _g_examples:
    _a = grid_to_np(_g)
    assert _a.ndim == 2 and _a.dtype == np.uint8


In [3]:

def discover_period(g, axis=1, max_p=None):
    H, W = g.shape
    max_p = max_p or CONFIG["MAX_PERIOD"]
    if axis == 1:
        for p in range(1, min(W, max_p)+1):
            if np.all(g == np.tile(g[:, :p], (1, W//p + 1))[:, :W]):
                return p
    else:
        for p in range(1, min(H, max_p)+1):
            if np.all(g == np.tile(g[:p, :], (H//p + 1, 1))[:H, :]):
                return p
    return None

def detect_stripes(g):
    h, w = g.shape
    horiz = discover_period(g, axis=1)
    vert  = discover_period(g, axis=0)
    return {
        "h_period": horiz or 0,
        "v_period": vert or 0,
        "is_hstripe": bool(horiz and h>=horiz and w%horiz==0),
        "is_vstripe": bool(vert and w>=vert and h%vert==0),
    }

def ccl_components(g, bg=None):
    if not SCIPY:
        return []
    bg = CONFIG["BG"] if bg is None else bg
    comps = []
    for c in range(10):
        if c == bg: continue
        mask = (g == c)
        if not mask.any(): continue
        lab, n = label(mask)
        for fid in range(1, n+1):
            m = (lab == fid)
            if m.sum() > 0:
                comps.append((m, c))
    return comps


In [4]:

class DSL:
    @staticmethod
    def identity(g): return g.copy()
    @staticmethod
    def rotate_90(g):  return np.rot90(g, 1)
    @staticmethod
    def rotate_180(g): return np.rot90(g, 2)
    @staticmethod
    def rotate_270(g): return np.rot90(g, 3)
    @staticmethod
    def flip_h(g): return np.fliplr(g)
    @staticmethod
    def flip_v(g): return np.flipud(g)
    @staticmethod
    def transpose(g): return g.T
    @staticmethod
    def shift_left(g):  return np.roll(g, -1, axis=1)
    @staticmethod
    def shift_right(g): return np.roll(g,  1, axis=1)
    @staticmethod
    def shift_up(g):    return np.roll(g, -1, axis=0)
    @staticmethod
    def shift_down(g):  return np.roll(g,  1, axis=0)

    @staticmethod
    def invert_colors(g):
        r = g.copy()
        r = np.where(r!=0, 10 - r, r)
        return r

    @staticmethod
    def replace_color(g, old, new):
        r = g.copy(); r[g==old] = new; return r

    @staticmethod
    def recolor_with_palette_bijection(g, ref):
        if not SCIPY: return g.copy()
        colors_in  = np.unique(g)
        colors_ref = np.unique(ref)
        cost = np.zeros((len(colors_in), len(colors_ref)))
        for i, ci in enumerate(colors_in):
            for j, cj in enumerate(colors_ref):
                overlap = np.sum((g==ci) & (ref==cj))
                cost[i, j] = -overlap
        ri, cj = linear_sum_assignment(cost)
        mapping = {int(colors_in[i]): int(colors_ref[j]) for i,j in zip(ri, cj)}
        out = g.copy()
        for cin, cout in mapping.items():
            out[g==cin] = cout
        for cin in set(colors_in)-set(mapping.keys()):
            out[g==cin] = CONFIG["BG"]
        return out

    @staticmethod
    def scale_2x(g): return np.repeat(np.repeat(g,2,axis=0),2,axis=1)
    @staticmethod
    def scale_3x(g): return np.repeat(np.repeat(g,3,axis=0),3,axis=1)
    @staticmethod
    def crop_to_content(g, bg=None):
        bg = CONFIG["BG"] if bg is None else bg
        mask = (g != bg)
        if not mask.any(): return g.copy()
        rows = np.any(mask, axis=1)
        cols = np.any(mask, axis=0)
        return g[rows][:, cols]
    @staticmethod
    def add_border(g, b=1, color=1):
        h,w = g.shape
        r = np.full((h+2*b, w+2*b), color, dtype=g.dtype)
        r[b:h+b, b:w+b] = g
        return r

    @staticmethod
    def extract_largest_object(g, bg=None):
        bg = CONFIG["BG"] if bg is None else bg
        comps = ccl_components(g, bg)
        if not comps: return g.copy()
        largest = max(comps, key=lambda x: x[0].sum())
        r = np.full_like(g, bg)
        r[largest[0]] = largest[1]
        return r

    @staticmethod
    def separate_objects_by_color(g, bg=None):
        bg = CONFIG["BG"] if bg is None else bg
        comps = ccl_components(g, bg)
        r = np.full_like(g, bg)
        for i,(m,c) in enumerate(comps):
            r[m] = (i+1) % 10
        return r

    @staticmethod
    def align_objects_grid(g, grid_size=3, bg=None):
        bg = CONFIG["BG"] if bg is None else bg
        comps = ccl_components(g, bg)
        if not comps: return g.copy()
        objs = []
        for m,c in comps:
            rs, cs = np.where(m)
            if len(rs)==0: continue
            objs.append((rs.min(), cs.min(), rs.max(), cs.max(), c, m))
        objs.sort(key=lambda x:(x[0],x[1]))
        h,w = g.shape; cell_h=h//grid_size; cell_w=w//grid_size
        r = np.full_like(g, bg)
        for i,(r0,c0,r1,c1,c,m) in enumerate(objs[:grid_size*grid_size]):
            if cell_h==0 or cell_w==0: break
            oh, ow = r1-r0+1, c1-c0+1
            gr, gc = i//grid_size, i%grid_size
            sr = gr*cell_h + max(0,(cell_h-oh)//2)
            sc = gc*cell_w + max(0,(cell_w-ow)//2)
            oh2 = min(oh, r.shape[0]-sr); ow2 = min(ow, r.shape[1]-sc)
            if oh2<=0 or ow2<=0: continue
            sub = m[r0:r0+oh2, c0:c0+ow2]
            r[sr:sr+oh2, sc:sc+ow2][sub] = c
        return r

    @staticmethod
    def tile_to(g, h, w):
        gh, gw = g.shape
        if gh==0 or gw==0: return g.copy()
        t = np.tile(g, (math.ceil(h/gh), math.ceil(w/gw)))
        return t[:h,:w]

def build_primitive_families(train_pairs):
    fam = {
        "geometric": {
            "identity": DSL.identity, "rotate_90": DSL.rotate_90, "rotate_180": DSL.rotate_180, "rotate_270": DSL.rotate_270,
            "flip_h": DSL.flip_h, "flip_v": DSL.flip_v, "transpose": DSL.transpose,
            "shift_left": DSL.shift_left, "shift_right": DSL.shift_right, "shift_up": DSL.shift_up, "shift_down": DSL.shift_down,
        },
        "color": {
            "invert": DSL.invert_colors,
            "recolor_bijection": (lambda g, ref=train_pairs[0]["output"]: DSL.recolor_with_palette_bijection(g, ref) if isinstance(ref,np.ndarray) else g),
        },
        "size": {
            "scale_2x": DSL.scale_2x, "scale_3x": DSL.scale_3x, "crop_to_content": DSL.crop_to_content, "add_border": DSL.add_border,
        },
        "object": {
            "extract_largest": DSL.extract_largest_object, "separate_by_color": DSL.separate_objects_by_color, "align_grid": DSL.align_objects_grid,
        },
        "pattern": {
            "tile_to_ref": (lambda g, ref=train_pairs[0]["output"]: DSL.tile_to(g, ref.shape[0], ref.shape[1]) if isinstance(ref,np.ndarray) else g)
        }
    }
    colors = set()
    for p in train_pairs:
        colors |= set(np.unique(p["input"])); colors |= set(np.unique(p["output"]))
    colors = sorted(list(colors))[:8]
    for a in colors:
        for b in colors:
            if a==b: continue
            name = f"replace_{a}_to_{b}"
            fam["color"][name] = (lambda g, aa=a, bb=b: DSL.replace_color(g, aa, bb))
    return fam


In [5]:

def induce_size_rule(train_pairs):
    dims_in  = [p["input"].shape for p in train_pairs]
    dims_out = [p["output"].shape for p in train_pairs]
    rh = []; rw = []; dh = []; dw = []
    for di,do in zip(dims_in,dims_out):
        if di[0]>0 and di[1]>0:
            if do[0]%di[0]==0: rh.append(do[0]//di[0])
            if do[1]%di[1]==0: rw.append(do[1]//di[1])
        dh.append(do[0]-di[0]); dw.append(do[1]-di[1])
    if rh and rw and len(set(rh))==1 and len(set(rw))==1:
        return {"type":"ratio","h":rh[0],"w":rw[0]}
    if dh and dw and len(set(dh))==1 and len(set(dw))==1:
        return {"type":"delta","h":dh[0],"w":dw[0]}
    return None

def try_fast_bijection_symmetry(train_pairs, test_input):
    SYMS = [lambda x:x, DSL.rotate_90, DSL.rotate_180, DSL.rotate_270, DSL.flip_h, DSL.flip_v, DSL.transpose]
    for s in SYMS:
        ok=True; mapping=None
        for pair in train_pairs:
            A = s(pair["input"]); B = pair["output"]
            if A.shape!=B.shape: ok=False; break
            m = {}
            for a,b in zip(A.flatten(), B.flatten()):
                if a in m and m[a]!=b: ok=False; break
                m[a]=b
            if not ok: break
            if mapping is None: mapping = m
            elif mapping != m: ok=False; break
        if ok and mapping is not None:
            T = s(test_input)
            R = T.copy()
            for a,b in mapping.items():
                R[T==a]=b
            return R
    return None


In [6]:

class TinyVetoNet(nn.Module if TORCH else object):
    def __init__(self, input_dim=32, hidden=64):
        if TORCH:
            super().__init__()
            self.net = nn.Sequential(
                nn.Linear(input_dim, hidden), nn.ReLU(),
                nn.Linear(hidden, hidden), nn.ReLU(),
                nn.Linear(hidden, 1), nn.Sigmoid()
            )
        else:
            pass
    def forward(self, x):
        if TORCH: return self.net(x)
        return 0.5

def veto_features(g_pred, g_in, g_ref):
    H,W = g_pred.shape
    pal_pred = len(np.unique(g_pred))
    pal_in   = len(np.unique(g_in))
    size_ok = 1.0 if (g_ref is not None and g_pred.shape==g_ref.shape) else 0.0
    bg_pred = float((g_pred==CONFIG["BG"]).mean())
    bg_in   = float((g_in  ==CONFIG["BG"]).mean())
    iou = float((g_pred==g_ref).mean()) if (g_ref is not None and g_pred.shape==g_ref.shape) else 0.0
    d = detect_stripes(g_pred)
    feats = np.array([H,W,pal_pred,pal_in,size_ok,bg_pred,bg_in,iou,
                      d["h_period"], d["v_period"], int(d["is_hstripe"]), int(d["is_vstripe"])], dtype=np.float32)
    if feats.size < CONFIG["VETO_FEATS"]:
        feats = np.pad(feats, (0, CONFIG["VETO_FEATS"]-feats.size))
    else:
        feats = feats[:CONFIG["VETO_FEATS"]]
    return feats.astype(np.float32)

@dataclass
class Node:
    grid: np.ndarray
    program: List[Tuple[str,str]] = field(default_factory=list)
    fam_counts: Dict[str,int] = field(default_factory=lambda: defaultdict(int))
    score: float = 0.0

def fit_score(pred, target):
    if pred.shape!=target.shape: return 0.0
    acc = (pred==target).mean()
    ha = Counter(pred.flatten()); hb = Counter(target.flatten())
    inter = sum(min(ha[k], hb[k]) for k in set(ha)|set(hb))
    hist = inter / max(1, pred.size, target.size)
    return 0.75*acc + 0.25*hist

def beam_search(task, veto_model):
    train = [{"input":grid_to_np(p["input"]), "output":grid_to_np(p["output"])} for p in task["train"]]
    test_in = grid_to_np(task["test"][0]["input"])
    fams = build_primitive_families(train)
    veto_ref = train[0]["output"] if train else None

    start = try_fast_bijection_symmetry(train, test_in)
    beam = [Node(start if start is not None else test_in)]
    best = (None, 0.0, [])

    t0 = time.time()
    for depth in range(CONFIG["BEAM_DEPTH"]):
        if time.time()-t0 > CONFIG["TIME_PER_TASK_S"]*0.8:
            log.warning("Beam budget nearing; breaking.")
            break

        cand = []
        for node in beam:
            for fam, ops in fams.items():
                if node.fam_counts.get(fam,0) >= CONFIG["FAMILY_CAPS"].get(fam,1):
                    continue
                for opname, op in ops.items():
                    try:
                        g2 = op(node.grid)
                        vscore = 1.0
                        if veto_model is not None and TORCH:
                            feats = veto_features(g2, test_in, veto_ref)
                            with torch.no_grad():
                                vscore = float(veto_model(torch.tensor(feats).unsqueeze(0)).item())
                        if vscore < CONFIG["VETO_THRESH"]:
                            continue
                        scs = [fit_score(g2, p["output"]) for p in train] if train else [0.5]
                        sc = min(scs) if CONFIG["RELAXED_OK"] else (1.0 if all(s==1.0 for s in scs) else 0.0)
                        new = Node(g2, node.program+[(fam,opname)], dict(node.fam_counts))
                        new.fam_counts[fam] = new.fam_counts.get(fam,0)+1
                        new.score = sc
                        cand.append(new)
                        if sc > best[1]:
                            best = (g2, sc, new.program)
                    except Exception:
                        continue
        cand.sort(key=lambda n: n.score, reverse=True)
        beam = cand[:CONFIG["BEAM_WIDTH"]]
        if best[1] >= 0.999:
            break

    return best


In [7]:

class TinyPolicy(nn.Module if TORCH else object):
    def __init__(self, action_dim):
        if TORCH:
            super().__init__()
            self.enc = nn.Sequential(
                nn.Conv2d(1,16,3,padding=1), nn.ReLU(),
                nn.Conv2d(16,32,3,padding=1), nn.ReLU(),
                nn.AdaptiveAvgPool2d((6,6)),
            )
            self.head = nn.Sequential(
                nn.Flatten(),
                nn.Linear(32*6*6, 128), nn.ReLU(),
                nn.Linear(128, action_dim)
            )
        else:
            pass
    def forward(self, x):
        if TORCH:
            return self.head(self.enc(x))
        return 0.0

def neural_policy_suggest(train_pairs, fams, steps=12):
    actions = [(fam,op) for fam in fams for op in fams[fam].keys()]
    if not TORCH:
        random.shuffle(actions); 
        return actions[:min(10,len(actions))]
    net = TinyPolicy(action_dim=len(actions))
    opt = optim.Adam(net.parameters(), lr=CONFIG["TTT_LR"])
    losses = []
    for _ in range(steps):
        if not train_pairs: break
        p = random.choice(train_pairs)
        x = torch.tensor(p["input"][None,None,:,:], dtype=torch.float32)
        y = torch.tensor(p["output"][None,None,:,:], dtype=torch.float32)
        feat_x = net.enc(x); feat_y = net.enc(y)
        loss = 1.0 - F.cosine_similarity(feat_x.flatten(1), feat_y.flatten(1)).mean()
        opt.zero_grad(); loss.backward(); opt.step()
        losses.append(float(loss.item()))
    if losses:
        from statistics import mean
        METRICS.log(policy_ttt_loss=mean(losses))
    if train_pairs:
        x = torch.tensor(train_pairs[-1]["input"][None,None,:,:], dtype=torch.float32)
        with torch.no_grad():
            logits = net(x).flatten().cpu().numpy()
        order = np.argsort(-logits)
    else:
        order = np.arange(len(actions))
    ranked = [actions[i] for i in order[:min(20, len(actions))]]
    return ranked


In [8]:

def solve_one_task(task, veto_model):
    t0 = time.time()
    train = safe_get_train(task)
    test_in = safe_get_test_input(task, 0)

    size_rule = induce_size_rule(train) if train else None
    if size_rule and size_rule["type"]=="ratio":
        H = test_in.shape[0]*size_rule["h"]; W = test_in.shape[1]*size_rule["w"]
        fast = DSL.tile_to(test_in, H, W)
        METRICS.log(fast_path="size_ratio", h=size_rule["h"], w=size_rule["w"])
        return {"attempt_1": np_to_grid(fast), "attempt_2": np_to_grid(fast), "confidence": 1.0, "source":"fast_ratio"}

    g_sym, c_sym, prog = beam_search(task, veto_model)
    METRICS.log(stage="symbolic", conf=round(c_sym,4), prog_len=(len(prog) if prog else 0))

    if g_sym is not None and c_sym >= 0.99:
        return {"attempt_1": np_to_grid(g_sym), "attempt_2": np_to_grid(g_sym), "confidence": float(c_sym), "source": "symbolic_exact"}

    fams = build_primitive_families(train)
    suggestions = neural_policy_suggest(train, fams, steps=CONFIG["TTT_STEPS"])
    node = Node(test_in); best = (g_sym, c_sym, prog if prog else [])
    for (fam,opname) in suggestions[:10]:
        try:
            g2 = fams[fam][opname](node.grid)
            scs = [fit_score(g2, p["output"]) for p in train] if train else [0.5]
            sc = min(scs) if CONFIG["RELAXED_OK"] else (1.0 if all(s==1.0 for s in scs) else 0.0)
            if sc > best[1]:
                best = (g2, sc, [(fam,opname)])
        except:
            pass

    g_best, c_best, prog_best = best
    src = "neural_guided" if prog_best and prog_best!=prog else "symbolic_loose"
    if g_best is None:
        g_best = test_in.copy(); c_best = 0.5; src="fallback_identity"
    alt = DSL.align_objects_grid(g_best) if SCIPY else g_best
    return {"attempt_1": np_to_grid(g_best), "attempt_2": np_to_grid(alt), "confidence": float(c_best), "source": src}

def chronos_orca_run(TEST, task_ids):
    veto = TinyVetoNet(32, 64) if TORCH else None
    out = {}
    for i, tid in enumerate(task_ids):
        log.info(f"=== Task {i+1}/{len(task_ids)}: {tid} ===")
        res = solve_one_task(TEST[tid], veto)
        out[tid] = [{"attempt_1": res["attempt_1"], "attempt_2": res["attempt_2"]}]
        METRICS.log(task=tid, conf=res["confidence"], source=res["source"])
    return out


In [9]:

def find_arc_paths():
    cands = ["/kaggle/input/arc-prize-2025","/kaggle/input/arc-agi","/kaggle/input/arc-prize-2024","."]
    files = {"test_ch": ["arc-agi_test_challenges.json","test.json","arc_test.json"]}
    out = {}
    for key, names in files.items():
        for d in cands:
            for nm in names:
                p = Path(d)/nm
                if p.exists():
                    out[key] = str(p); break
            if key in out: break
    return out

def load_TEST(path):
    with open(path,"r") as f:
        data = json.load(f)
    for k,task in data.items():
        for pair in task.get("train",[]):
            pair["input"]  = np.array(pair["input"], dtype=np.uint8)
            pair["output"] = np.array(pair["output"], dtype=np.uint8)
        for pair in task.get("test",[]):
            pair["input"]  = np.array(pair["input"], dtype=np.uint8)
    return data, list(data.keys())

def write_submission(sub, path="submission.json"):
    with open(path,"w") as f:
        json.dump(sub, f)
    log.info(f"Saved submission to {path} ({os.path.getsize(path)} bytes)")


In [10]:

if __name__ == "__main__":
    paths = find_arc_paths()
    if "test_ch" not in paths:
        log.error("Test challenge JSON not found. Place ARC dataset in /kaggle/input.")
    else:
        TEST, ids = load_TEST(paths["test_ch"])
        log.info(f"Loaded {len(ids)} test tasks from {paths['test_ch']}")
        submission = chronos_orca_run(TEST, ids)
        write_submission(submission, "submission.json")
        log.info("Metrics summary: %s", METRICS.summary())



## Knobs & Dials
- `TIME_PER_TASK_S`, `BEAM_WIDTH`, `BEAM_DEPTH`, `FAMILY_CAPS` — search controls
- `RELAXED_OK`, `RELAXED_MIN_IOU` — relaxed fit mode
- `VETO_THRESH`, `VETO_FEATS`, `VETO_HIDDEN` — veto net behavior
- `TTT_STEPS`, `TTT_LR` — tiny policy adaptation

## Design Notes
- Contest-safe: no I/O beyond dataset and submission.json; no network.
- Verbose logs: per-task metrics line, final summary.
- Extensible: add more DSL ops or a bigger policy net within resource limits.



## Diagnostics & Self-Check
The following utilities help validate solver components offline:

- **Palette bijection property tests**  
- **CCL invariants**: area conservation for masks  
- **Stripe detector sanity checks**  
- **Beam search smoke tests**  
- **Neural policy shape checks**  


In [11]:

# Self-checks (non-exhaustive). Safe to run locally.

def _test_palette_bijection():
    a = np.array([[1,2],[3,4]])
    b = np.array([[4,3],[2,1]])
    out = DSL.recolor_with_palette_bijection(a,b) if 'linear_sum_assignment' in globals() else a
    assert out.shape == a.shape

def _test_ccl_invariants():
    g = np.array([[0,1,1,0],[0,1,0,2],[0,0,2,2]])
    comps = ccl_components(g)
    total = 0
    for m,c in comps:
        total += int(m.sum())
    assert total == int((g!=0).sum())

def _test_stripes():
    g = np.array([[1,2,1,2],[1,2,1,2]])
    d = detect_stripes(g)
    assert d["h_period"] in (0,2,4)

def _test_beam_smoke():
    task = {
        "train":[{"input": [[1,1],[2,2]], "output": [[2,2],[1,1]]}],
        "test":[{"input": [[3,3],[4,4]]}]
    }
    task["train"][0]["input"] = np.array(task["train"][0]["input"],dtype=np.uint8)
    task["train"][0]["output"] = np.array(task["train"][0]["output"],dtype=np.uint8)
    task["test"][0]["input"] = np.array(task["test"][0]["input"],dtype=np.uint8)
    g, c, p = beam_search(task, None)
    assert isinstance(c, float)

def run_self_checks():
    _test_palette_bijection()
    _test_ccl_invariants()
    _test_stripes()
    _test_beam_smoke()
    print("Self-checks passed.")

# Uncomment to run in a local environment (not on ARC eval)
# run_self_checks()



## Extended Design Rationale

### Symbolic First
ARC tasks overwhelmingly decompose to short programs over geometry, color, tiling, and object logic.
A beam over a broad but **bounded** DSL is the most sample-efficient first strike.

### Neural Where It Helps
A tiny policy is used only to **rank** candidate operations and provide light **TTT adaptation**
to the specifics of a task's palette and motifs (no external data).

### Hybrid Verification
Whenever possible, we prefer symbolically verifiable solutions (exact program reproducing train pairs),
and only accept raw neural predictions when verification fails and confidence is still adequate.

### Metrics-First Engineering
The engine prints a compact, parseable metrics line per task and an aggregate summary at the end.
This keeps iterations fast and scientific without external tooling.

### Safe Defaults & Contest Compliance
- No internet or external calls.
- Exactly two attempts per test grid written in the `submission.json` per ARC rules.
- Deterministic seed initialization.

### Roadmap Hooks (Commented)
- Wider family of DSL ops (morphology, flood-fill edits, symmetry group closures).
- Better veto features (entropy, spectrum of Laplacian, object aspect histograms).
- Learned priors over program lengths and family orderings.
